# About 
This is a general procedure of how to prepare the data. Directories and File names will be kept empty, but all the functions used here are as used in the paper.
This script can be used for both the fiducial statistics and the varied parameters - though the form has their LoS evolution summarised.

You will need to run: _pip install pywst_, in order to use RWST. 

In [ ]:
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
from astropy.io import fits
from os import listdir
import os
from scipy.fftpack import fft, dct
import sys
sys.path.insert(0, os.path.abspath('..')) # Note that this line is useless with a regular pip installation of PyWST.
import pywst as pw
ddir = './'
import pywt
%matplotlib inline

In [ ]:
def PS_2D_WF_Load(dat, box_size, npix, nbins,Binned_WF_PS):
    """ Calculates the 2D Power spectrum, using a given window function. 
        The inpit should be the same as that given to calculate Binned_WF_PS. 

    Parameters
    ----------
    dat : numpy.array
        2D array of a frequency slice of a lightcone or coeval cube
    box_size : float
       2D extend of simulation, in Mpc.
    npix : int
        The number of pixels alone wither the x or y direction.
    nbins : int
        The number of bins to be used for the power spectrum
    Binned_WF_PS : nump.array
        The binning window function.
    
    Returns
    -------
    binned_data : numpy.array
        2D power spectrum containing nbins
    """
    import numpy as np
    from numpy import fft as f 
    from astropy.io import fits
    import os 
    import matplotlib.pyplot as plt
    from astropy.cosmology import FlatLambdaCDM,WMAP9
    import astropy.units as u 
    import numpy as np
    from numpy import fft as f 
    import math 
    from astropy.io import fits
    import matplotlib.pyplot as plt
    import sys
    
    # Calculate box dimensions in k for binning 
    box_size = float(box_size)
    k_small_perp = (2 * np.pi) / box_size
    k_large_perp = npix * k_small_perp
    H_0= WMAP9.H0.value
    k_small = k_small_perp
    k_largest = k_large_perp
    bins = np.logspace(math.log10(k_small), math.log10(k_largest), nbins + 1)
    dk_xy = (box_size / npix) / (2 * np.pi)
    nu_xy = np.fft.fftshift(np.fft.fftfreq(npix, dk_xy))
    kx, ky = np.meshgrid(nu_xy, nu_xy)
    kmod = np.sqrt(kx ** 2 + ky ** 2)
    kernel_1 = []
    kernel_2 = []
    k = np.zeros(nbins)
    # Create bins 
    for p in range(len(bins) - 1):
        c1 = bins[p] <= kmod
        c2 = kmod[c1] < bins[p + 1]
        kernel_1.append(c1)
        kernel_2.append(c2)
        k[p] = bins[p + 1] * 100 / H_0
    # Power spectrum set up     
    dat_2D = dat
    ft2d = f.fftn(dat_2D) / np.int(dat_2D.size)
    ft2d = f.fftshift(ft2d)
    PS2D = np.abs(ft2d) ** 2
    ps = np.array(PS2D)
    binned_data = np.zeros(nbins)  # The binning of the FT
    ps_err = np.zeros(nbins)
    # Bin Power spectrum
    for bin_num in range(len(bins) - 1):
        kmod_1 = abs(kmod[kernel_1[bin_num]])
        kmod_2 = abs(kmod_1[kernel_2[bin_num]])
        Binned_PS = Binned_WF_PS[bin_num]
        weighted = ps*pow(Binned_PS,2)
        GG.append(Binned_PS)
        bin_data = np.sum(ps*(Binned_PS))/np.sum(Binned_PS)
        prefac = (pow(box_size, 2) / (2 * (np.pi) ** 2))
        binned_data[bin_num] = (prefac * (bin_data) * k[bin_num] ** 2)
        PSerror = np.var(ps*(Binned_PS))/np.sum(Binned_PS)
        ps_err[bin_num] = PSerror
    return binned_data#, k, ps_err,GG,ps

In [ ]:
def PS_3D_WF_Load(dat,box_size,npix,zmin,zmax,nbins,Binned_WF_PS):
    """ Calculates the 3D Power spectrum, using a given window function. 
        The inpit should be the same as that given to calculate Binned_WF_PS. 

    Parameters
    ----------
    dat : numpy.array
        3D array being the lightcone or coeval cube
    box_size : float
       2D extend of simulation, in Mpc.
    npix : int
        The number of pixels alone wither the x or y direction.
    zmin : float
        Minimum redshift of lightcone/coeval
    zmax : float
        Maximum redshift of lightcone/coeval
    nbins : int
        The number of bins to be used for the power spectrum
    Binned_WF_PS : nump.array
        The binning window function.
    
    Returns
    -------
    binned_data : numpy.array
        2D power spectrum containing nbins
    """
    import numpy as np
    from numpy import fft as f 
    from astropy.io import fits
    import os 
    import matplotlib.pyplot as plt
    from astropy.cosmology import FlatLambdaCDM,WMAP9
    import astropy.units as u 
    import numpy as np
    from numpy import fft as f 
    import math 
    from astropy.io import fits
    import matplotlib.pyplot as plt
    import sys 
    # Calculate box dimensions in k for binning 
    box_size = float(box_size)
    H_0= WMAP9.H0.value
    omega_m = WMAP9.Om0
    n_slices = (dat.shape)[0]
    dmin  = WMAP9.comoving_distance(zmin)
    dmax  = WMAP9.comoving_distance(zmax)
    delta_d = abs(dmax-dmin)
    k_small_perp = (2*np.pi)/box_size 
    k_large_perp = npix*k_small_perp #length of box side in k-space 
    k_small_los = (2*(np.pi)/delta_d)*u.Mpc
    k_large_los = k_small_los*n_slices 
    # Setting max and min k
    if k_small_los<k_small_perp:
        k_small = k_small_los
    else:
        k_small = k_small_perp    
    if k_large_los<k_large_perp:
        k_largest = k_large_perp
    else:
        k_largest = k_large_los
    bins = np.logspace(math.log10(k_small),math.log10(k_largest ), nbins+1)
    dk_xy = (box_size/npix)/(2*np.pi)
    dk_z = (delta_d/n_slices)/(2*np.pi)
    nu_xy = np.fft.fftshift(np.fft.fftfreq(npix, dk_xy)) 
    nu_z = np.fft.fftshift(np.fft.fftfreq(n_slices, dk_z)) 
    kx, ky = np.meshgrid(nu_xy,nu_xy)
    cube_z = np.zeros((len(nu_z),kx.shape[0],kx.shape[1]))
    for o,c in zip(nu_z,range(len(nu_z))):
        cube_z[c,:,:] = o
    cube_z = np.array(cube_z)
    # Power spectrum set up
    dat_3D = dat
    ft3d = f.fftn(dat_3D)/np.int(dat_3D.size)
    ft3d = f.fftshift(ft3d)
    PS3D = np.abs(ft3d)**2  
    ps = np.array(PS3D) 
    # Creating bins 
    kmod = np.zeros((cube_z.shape[0],cube_z.shape[1],cube_z.shape[2]))
    for p in range(cube_z.shape[0]):
        kmod[p,:,:] = np.sqrt(kx**2 + ky**2 + cube_z[p]**2)
    kernel_1 = []
    kernel_2 = []
    k = np.zeros(nbins)
    for p in range(len(bins)-1):
        c1 = bins[p] <=kmod
        c2 =kmod[c1]< bins[p+1] 
        kernel_1.append(c1)
        kernel_2.append(c2)
        k[p] = bins[p+1]*100/H_0
    binned_data =np.zeros(nbins,dtype=object) #The binning of the FT
    WF_Bins =np.zeros(nbins,dtype=object) #The binning of the FT
    binned_wf =np.zeros(nbins,dtype=object) #The binning of the FT
    ps_err=np.zeros(nbins)
    #Binning
    for bin_num in range(len(bins) - 1):
        kmod_1 = abs(kmod[kernel_1[bin_num]])
        kmod_2 = abs(kmod_1[kernel_2[bin_num]])        

        WF_Bins[bin_num] = Binned_WF_PS[bin_num]
        weighted = ps*abs(pow(Binned_WF_PS[bin_num],2))
        binned_wf[bin_num] = weighted
        bin_data = np.sum(ps*(abs(pow(Binned_WF_PS[bin_num],2))))/np.sum(abs(pow(Binned_WF_PS[bin_num],2)))
        volme = (box_size*box_size*delta_d)/u.Mpc
        prefac = (volme/(2*(np.pi)**2))   
        binned_data[bin_num] = (prefac*((bin_data))*k[bin_num]**3) 
        PSerror = np.var(ps*(abs(pow(Binned_WF_PS[bin_num],2))))/np.sum(abs(pow(Binned_WF_PS[bin_num],2)))
        ps_err[bin_num] = PSerror
    return binned_data#,k,WF_Bins,binned_wf

In [ ]:
def Inv_WST_Load(dat,nbins,Binned_ft2d_WF):
    """ Calculates the 2D wavelet moments of a 2D field

    Parameters
    ----------
    dat : numpy.array
        2D array of a frequency slice of a lightcone or coeval cube
    nbins : int
        The number of bins to be used for the power spectrum
    Binned_WF_PS : nump.array
        The binning window function.

    
    Returns
    -------
    L1_ : numpy.array
        2D Wavelet moments summarised with the L1-norm
    L2_ : numpy.array
        2D Wavelet moments summarised with the L2-norm
    """
    
    import numpy as np
    from numpy import fft as f 
    binned_data = np.zeros(nbins,dtype=object)  # The binning of the FT
    L2_ = np.zeros(nbins,dtype=object) 
    L1_ = np.zeros(nbins,dtype=object) 
    for bin_num in range(nbins - 1):
        Binned_ft2d = Binned_ft2d_WF[bin_num]
        shifted_weighted = f.ifftshift(ft2d*Binned_ft2d)
        Wavelet_Conv = f.ifftn(shifted_weighted)
        L2_[bin_num] = np.linalg.norm(Wavelet_Conv) 
        L1_[bin_num] =np.sum(np.abs(Wavelet_Conv))
    return L1_,L2_

## Set Parameters 

In [ ]:
params = ['ION_Tvir_MIN','R_BUBBLE_MAX','HII_EFF_FACTOR']
nparams = len(params)
max_samples = #Maximum number of samples you want to use 
npix = # 2D pixel extent of the coeval cube or lightcone you want to use
box_size = # 2D extent of the coeval cube or lightcone you want to use
freq_channels = # Number of frequency channels in the coeval cube or lightcone you want to use
nbins = #Number of bins you want to use for your power spectrum 
zmin,zmax= #Min. z of coeval cube or lightcone, #Max. z of coeval cube or lightcone
M,N=npix,npix
J = # We choose number of scales to probe
L = # Number of angles in which the interval [0,pi] is divided
Total_Scales = J + np.arange(J).sum()#S1Iso, S2Iso1
OS = 0 # No oversampling
wst_op = pw.WSTOp(M, N, J, L, OS)
rwst_op = pw.RWSTOp(M, N, J, L, OS)

In [ ]:
#Parameter to run for
param = params[1]


In [ ]:
# The loaded lightcone should contain all samples and have dimensions: [namples,nfreq_channels,npix,npix]
# This is loaded for a particular parameter's +/-, to be used for the derivatives, or for the fiducial model,
# to be used for the covariance 
LightCones = # Load lightcone 


## Getting the window functions

The procedure for generating the window functions is as follows. We fist generate our log-bins:

<code> np.logspace(math.log10(k_small),math.log10(k_largest ), nbins+1) </code>

Next, for our 2D power spectrum or wavelet moments, we can define our 2D wavenumber space:

  
<code>dk_xy = (box_size / npix) / (2 * np.pi)
nu_xy = np.fft.fftshift(np.fft.fftfreq(npix, dk_xy))
kx, ky = np.meshgrid(nu_xy, nu_xy)
ft2d = np.array(ft2d)
kmod = np.sqrt(kx ** 2 + ky ** 2)<code>

To get each bin, we can produce masks:

<code>kernel_1 = []
kernel_2 = []
k = np.zeros(nbins)
for p in range(len(bins) - 1):
    c1 = bins[p] <= kmod
    c2 = kmod[c1] < bins[p + 1]
    kernel_1.append(c1)
    kernel_2.append(c2)
    k[p] = bins[p + 1] * 100 / H_0 <code>
    
With this, we can use kmod to produce each of our window functions for each bin. We will use
a 1D Gaussian, defined as:

In [1]:
def gaussian_1d(x, mu, sigma):
    """
    Compute the value of a 1D Gaussian function at a given x coordinate.
    
    Parameters:
    - x: The x coordinate at which to evaluate the Gaussian function.
    - mu: The mean value of the Gaussian distribution.
    - sigma: The standard deviation of the Gaussian distribution.
    
    Returns:
    - The value of the Gaussian function at x.
    """
    #coeff = 1 / (sigma * np.sqrt(2 * np.pi))
    exponent = -((x - mu) ** 2) / (2 * sigma ** 2)
    #return coeff * np.exp(exponent)
    return  np.exp(exponent)

We produce the window functions we use: 
<code>
    for bin_num in range(len(bins) - 1):
    kmod_1 = abs(kmod[kernel_1[bin_num]])
    kmod_2 = abs(kmod_1[kernel_2[bin_num]])
    WF = np.zeros((npix,npix))
    for i in range(npix):
        for j in range(npix):
             current_k = kmod[i,j]
             G = gaussian_1d(current_k,np.mean(kmod_2),np.std(kmod_2))
             WF[i,j] = G<code>
    
WF can now be saved as your window function and can be used in PS_2D_WF_Load. 
This procedure can be easily extended to the 3D. 

## Getting Stats

In [ ]:
# Initialise stats
PS_Evol= np.zeros((max_samples,freq_channels,nbins))
RWST_Evol= np.zeros((max_samples,freq_channels,Total_Scales))
PS_3D= np.zeros((max_samples,nbins))
WM_L2= np.zeros((max_samples,freq_channels,nbins))
WM_L1= np.zeros((max_samples,freq_channels,nbins))


# Load window functions 
WF_Bins_2D_Loaded = 
WF_Bins_3D_Loaded = 
for j,Sample in enumerate(LightCones):
    # Spherically averaged PS
    PS_3D[j,:] = PS_3D_WF_Load(Sample,box_size,npix,zmin,zmax,nbins,WF_Bins_3D_Loaded)
    # Go through each frequency slice 
    for k,data in enumerate(Sample):
        # 2D PS
        PS_Evol[j,k,:] = PS_2D_WF_Load(data,box_size,npix,nbins,WF_Bins_2D_Loaded)
        # 2D WM
        WM_L1[j,k,:],WM_L2[j,k,:] = Inv_WST_Load(data,box_size,npix,nbins,WF_Bins_2D_Loaded)
        # RWST
        rwst_app = rwst_op.apply(data)
        s1iso = rwst_app.get_coeffs(name = 'S1Iso')
        s2iso1_full = rwst_app.get_coeffs(name = 'S2Iso1').reshape(-1)
        # RWST outputs a square array, but for j2<j1 we will have 0. So we flatten and remove the 0s.
        s2iso1_nozeros = s2iso1_full[s2iso1_full!=0]
        RWST_Evol[j,k,:] = np.concatenate((s1iso,s2iso1_nozeros))  
        
np.savez(ddir+'PS_2D_WF_'+param+'_Evolution.npz',PS_Evol,dtype=object,allow_pickle=True)
np.savez(ddir+'WM_WF_L1_'+param+'_Evolution.npz',WM_L1,dtype=object,allow_pickle=True)
np.savez(ddir+'WM_WF_L2_'+param+'_Evolution.npz',WM_L2,dtype=object,allow_pickle=True)
np.savez(ddir+'RWST_Full_'+param+'_Evolution.npz',RWST_Evol,dtype=object,allow_pickle=True)
np.savez(ddir+'PS_3D_WF_'+param+'.npz',PS_3D_spherical,dtype=object,allow_pickle=True)


### For Fiducial

In the Fisher notebook, our Fiducial stats already have their LoS evolution summarised by the following functions:

In [ ]:
def LoS_Decomp_L2(Data,coefficients,max_samples=600):
    """ Here we look to summarise the line-of-sight information of a given statistics 
        coefficients using the \ell^2-norm. We then use this to calculate the derivatives 
        of this statistic, to be used in the Fisher analysis.
    Parameters
    ----------
    Data : numpy.array
        An array containing the statistics of the fiducial simulations, with dimensions of 
    coefficients : int
        Number of coefficients in the statistics
    nparams : int
        The number of parameters for which a derivative has been calculated.
    max_samples : int (default = 600)
        The maximum number of samples in the arrays
    
    Returns
    -------
    FID : numpy.array        
    """
    # The 2^j scales we can to decompose our statistic into 
    l2_scales = [pow(2,1),pow(2,2),pow(2,3),pow(2,4)]
    FID = np.zeros((max_samples,(coefficients),len(l2_scales)+1))
    # Consider a single parameter 
    for j in range(max_samples):
        # Look at each coefficients LoS evolution
        for k in range(coefficients):
            Bin_Evolution = Plus_param[j,:,k]
            cwtmatr, _ = pywt.cwt(Bin_Evolution, l2_scales, 'morl') 
            for l in range(len(cwtmatr)):
                # Calculate the \ell^2-norm of the evolution and then use it for the derivative 
                FID[j,k,l] =  np.linalg.norm(cwtmatr[l])
                # Don't forget the mean of the evolution!
                FID[j,k,l+1] = np.mean(Bin_Evolution)
    return FID

def LoS_Decomp_L1L2(Data,coefficients,max_samples=600):
    """ Here we look to summarise the line-of-sight information of a given statistics 
        coefficients using the \ell^2-norm. We then use this to calculate the derivatives 
        of this statistic, to be used in the Fisher analysis.
    Parameters
    ----------
    Data : numpy.array
        An array containing the statistics of the fiducial simulations, with dimensions of 
    coefficients : int
        Number of coefficients in the statistics
    nparams : int
        The number of parameters for which a derivative has been calculated.
    max_samples : int (default = 600)
        The maximum number of samples in the arrays
    
    Returns
    -------
    FID : numpy.array        
    """
    # The 2^j scales we can to decompose our statistic into 
    l1l2_scales = [pow(2,1),pow(2,2)]
    FID = np.zeros((max_samples,(coefficients),len(l1l2_scales)+1))
    # Consider a single parameter 
    for j in range(max_samples):
        # Look at each coefficients LoS evolution
        for k in range(coefficients):
            Bin_Evolution = Plus_param[j,:,k]
            cwtmatr, _ = pywt.cwt(Bin_Evolution, l1l2_scales, 'morl') 
            for l in range(len(cwtmatr)):
                # Calculate the \ell^1-norm and \ell^2-norm of the evolution and then use it for the derivative 
                FID[i,j,k,len(l1l2_scales)*l] =  np.linalg.norm(cwtmatr[l])
                FID[i,j,k,(len(l1l2_scales)*l)+1] =  np.sum(np.abs(cwtmatr[l]))
            # Don't forget the mean of the evolution!
            FID[i,j,k,-1] = np.mean(Bin_Evolution)
    return FID
  

In [ ]:
#Example of how to use:
PS_Evol_L1L2 = LoS_Decomp_L1L2(PS_Evol,9)
